헬스케어 데이터 분석에서 피할 수 없는 숙명인 '데이터 불균형(Data Imbalance)' 문제를 다룬다.

이 시간의 핵심은 "환자가 1%밖에 없는 상황에서 어떻게 AI에게 환자의 특징을 가르칠 것인가?"이다. 데이터를 인위적으로 조작(Sampling)하거나 모델에게 벌점(Weight)을 주는 방식을 통해, 소수 클래스(Minority Class)를 무시하지 않는 공정한 모델을 만드는 법을 배운다.

---

# 📘 [2일차 6교시] 6.1. 데이터 불균형 처리: 사막에서 바늘 찾기

## 1. 헬스케어 데이터의 불편한 진실: 불균형(Imbalance)

### 1) 교과서와 현실의 차이
머신러닝 교과서에 나오는 데이터들은 대부분 예쁘게 정리되어 있다. 개 사진 50장, 고양이 사진 50장처럼 정답의 비율이 5:5로 균형을 이룬다. 하지만 우리가 마주할 병원 현장의 데이터는 절대 그렇지 않다.

*   건강검진: 수만 명이 검사를 받지만, 암이 발견되는 사람은 극소수이다. (정상 99% vs 암 1%)
*   중환자실: 심정지(Cardiac Arrest) 예측 모델을 만든다고 하자. 다행히도 대부분의 시간 동안 환자의 심장은 뛴다. 심정지가 오는 순간은 아주 찰나이다. (정상 99.9% vs 심정지 0.1%)
*   부정 청구 탐지: 대부분의 병원은 정직하게 보험금을 청구한다. 사기꾼은 소수다.

이처럼 우리가 찾아내야 할 중요한 클래스(Positive, 1)는 매우 적고, 관심 없는 클래스(Negative, 0)는 압도적으로 많은 상황을 '데이터 불균형(Data Imbalance)'이라고 한다.

### 2) 왜 불균형이 문제가 되는가? (다수결의 횡포)
머신러닝 모델은 기본적으로 "전체 오차를 줄이고, 정확도(Accuracy)를 높이는 방향"으로 학습한다. 바로 이 지점에서 문제가 발생한다.

상황: 100명의 환자 데이터가 있고, 그중 단 1명만 암 환자이다.
모델은 학습 과정에서 고민에 빠진다. "저 1명의 암 환자를 찾기 위해 복잡한 패턴을 공부해야 할까?"
이때 모델은 게으른 꼼수를 발견한다.
> "그냥 고민하지 말고, 100명 다 '정상'이라고 찍어버리자."

이렇게 '모두 정상'이라고 찍었을 때의 성적표를 보자.
*   정상인 99명 $\rightarrow$ 정상 판정 (정답)
*   암 환자 1명 $\rightarrow$ 정상 판정 (오답)
*   정확도: 99점.

모델 입장에서는 99점이라는 훌륭한 점수를 받았다. 하지만 의사 입장에서는 암 환자를 놓쳤으므로 0점짜리 모델이다.
데이터가 불균형하면 모델은 소수 클래스(암 환자)를 '노이즈(Noise)'나 '예외 상황' 취급하며 무시해버린다. 결과적으로 다수 클래스(정상인)에만 과도하게 편향된 모델이 탄생한다. 이를 해결하기 위해 우리는 인위적인 개입을 해야 한다.

---

## 2. 해결책 1: 데이터를 바꿔라 (Resampling)

모델이 편식하지 않게 밥상(데이터)을 다시 차려주는 방법이다. 비율이 맞지 않는 두 그룹의 균형을 맞추기 위해, 데이터를 더하거나 빼는 작업을 '리샘플링(Resampling)'이라고 한다.

### 1) 언더 샘플링 (Under-sampling): 다수를 쳐내기
가장 단순한 방법이다. 비율이 맞을 때까지 '많은 쪽(다수 클래스) 데이터를 무작위로 삭제'하는 것이다.
*   방법: 정상인 99명, 환자 1명이라면 $\rightarrow$ 정상인 중 1명만 남기고 98명을 버린다. (1:1 비율)
*   장점: 데이터의 크기가 확 줄어드니 학습 속도가 엄청나게 빠르다.
*   단점 (치명적): "정보의 손실(Information Loss)".
    *   우리가 버린 98명의 정상인 데이터 속에, 정상인을 규정하는 아주 중요한 특징이 들어있었을지도 모른다.
    *   데이터가 수십만 건 이상으로 넘쳐나서 좀 버려도 상관없는 경우가 아니면, 헬스케어에서는 잘 쓰지 않는다.

### 2) 오버 샘플링 (Over-sampling): 소수를 불리기
부족한 '적은 쪽(소수 클래스) 데이터를 복제'해서 늘리는 것이다.
*   방법: 정상인 99명, 환자 1명이라면 $\rightarrow$ 환자 1명의 데이터를 99번 복사해서 99명으로 만든다. (1:1 비율)
*   장점: 정보 손실이 없다.
*   단점: "과적합(Overfitting)".
    *   모델이 똑같은 환자 데이터를 99번 반복해서 본다.
    *   결국 모델은 그 환자의 특징을 일반화하는 게 아니라, 그 환자 개인을 달달 외워버린다.
    *   새로운 유형의 암 환자가 오면 "내가 외운 그 사람이 아니네?" 하고 못 알아볼 수 있다.

이 단순 복제의 문제를 해결하기 위해 등장한 똑똑한 알고리즘이 바로 SMOTE이다.

---

## 3. SMOTE: 가상의 환자를 창조하라

SMOTE (Synthetic Minority Over-sampling Technique)는 단순히 데이터를 복사(Ctrl+C, Ctrl+V)하는 것이 아니라, 기존 데이터를 바탕으로 '새로운 가상의 데이터'를 만들어내는 기법이다.

### 1) 작동 원리: 점과 점 사이 잇기
SMOTE의 논리는 다음과 같다.
> "여기 암 환자 A가 있고, 저기 또 다른 암 환자 B가 있다. 이 둘은 서로 비슷하게 생겼다. 그렇다면 A와 B 사이의 어딘가(중간 지점)에 있는 가상의 존재 C도 암 환자이지 않을까?"

구체적인 과정은 다음과 같다.
1.  소수 클래스(환자) 중 하나(A)를 선택한다.
2.  그 환자와 가장 비슷하게 생긴(거리가 가까운) 이웃 환자(B)를 찾는다. (KNN 알고리즘 활용)
3.  A와 B를 연결하는 선을 긋는다.
4.  그 선 위의 무작위 지점에 점을 찍어 새로운 데이터(C)를 만든다.

### 2) SMOTE의 효과
*   단순히 똑같은 점이 겹치는 게 아니라, 환자들의 분포 영역 자체가 촘촘해지고 넓어진다.
*   모델은 특정 환자 한 명을 외우는 게 아니라, "이 영역에 분포하면 환자구나"라는 패턴(Decision Boundary)을 학습하게 된다.
*   과적합 위험을 줄이면서 데이터 불균형을 해소할 수 있어, 헬스케어 분야의 표준(Standard)처럼 사용되는 기법이다.

---

## 4. 해결책 2: 모델을 바꿔라 (Class Weight)

데이터를 조작하는 것이 찜찜할 수 있다. "없는 환자를 만들어내다니, 의학적으로 타당한가?"라는 비판이 있을 수 있기 때문이다. 이럴 때는 데이터는 그대로 두고, 모델의 '채점 기준(벌점)'을 바꾼다.

### 1) 가중치(Weight) 적용의 원리
기본적인 머신러닝 모델은 정상인을 틀리든 환자를 틀리든 똑같이 '벌점 1점'을 부과한다. 하지만 우리는 환자를 틀리는 것이 훨씬 심각하다는 것을 안다. 그래서 규칙을 바꾼다.

*   기본 규칙: 모든 오답은 벌점 1점.
*   가중치 규칙:
    *   정상인을 환자로 오진하면(FP) $\rightarrow$ 벌점 1점.
    *   환자를 정상으로 놓치면(FN) $\rightarrow$ 벌점 100점!

### 2) 모델의 변화
이렇게 세팅하고 학습을 시키면 모델은 공포에 떤다.
"정상인은 좀 틀려도 되지만, 환자는 절대 놓치면 안 돼. 점수가 다 깎여버리니까."
그래서 모델은 소수 클래스(환자)를 분류하는 데 온 신경을 집중하게 된다. 결과적으로 데이터 수는 적어도, 그 데이터가 가지는 영향력(Weight)이 커져서 균형이 맞춰지게 된다.

### 3) 구현 방법
Scikit-learn의 대부분의 모델(로지스틱 회귀, 랜덤 포레스트, SVM 등)은 `class_weight='balanced'`라는 옵션을 제공한다.
이 옵션을 켜기만 하면, 알아서 데이터 비율을 계산해 적은 쪽에는 높은 가중치를, 많은 쪽에는 낮은 가중치를 부여한다. 가장 간편하고 부작용이 적은 방법이다.

---

## 5. 요약: 불균형 데이터를 다루는 분석가의 자세

1.  현실 직시: 헬스케어 데이터는 원래 불균형하다. 정확도 99%에 속지 말고, 오차 행렬과 F1-Score를 확인해야 한다.
2.  데이터 관점 해결 (Resampling):
    *   데이터가 너무 많아서 처치 곤란이면 $\rightarrow$ 언더 샘플링 (잘 안 씀).
    *   데이터가 귀하면 $\rightarrow$ 오버 샘플링.
    *   단순 복제보다는 SMOTE를 사용하여 데이터의 분포를 채워주는 것이 좋다.
3.  모델 관점 해결 (Class Weight):
    *   데이터 조작 없이 모델에게 가중치(벌점)를 다르게 부여하여 소수 클래스에 집중하게 한다.
4.  주의사항: SMOTE 같은 오버 샘플링은 반드시 '학습 데이터(Train Set)'에만 적용해야 한다. 테스트 데이터(Test Set)는 현실 세계의 분포를 그대로 반영해야 하므로 절대 건드리면 안 된다. (모의고사는 쉽게 변형해서 공부해도 되지만, 수능 시험지를 조작하면 안 된다.)

## 6.2. \[예제 1\] 불균형 데이터의 시각화와 바보 모델의 탄생

데이터가 9:1로 불균형할 때, 모델이 소수 클래스를 어떻게 무시하는지
눈으로 확인합니다. (실습을 위해 `imbalanced-learn` 라이브러리가
필요합니다. Colab에는 기본 설치되어 있지 않을 수 있습니다.)

``` python
# 필요 라이브러리 설치
!pip install imbalanced-learn

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# 1. 극도로 불균형한 데이터 생성 (95% : 5%)
# weights=[0.95, 0.05]: 클래스 0은 95%, 클래스 1은 5%
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0,
                           n_clusters_per_class=1, weights=[0.95, 0.05], random_state=42)

# 2. 데이터 분포 시각화
plt.figure(figsize=(8, 6))
# 0(정상)은 파란색, 1(환자)은 빨간색
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, style=y, palette=['blue', 'red'], s=50)
plt.title("Imbalanced Data Distribution (95% vs 5%)")
plt.show()

# 3. 일반 모델 학습 (아무 처리 없음)
model = LogisticRegression()
model.fit(X, y)
y_pred = model.predict(X)

# 4. 결과 확인 (오차 행렬)
print("=== 일반 모델 성능 ===")
print(confusion_matrix(y, y_pred))
print(classification_report(y, y_pred))

# 설명:
# 시각화를 보면 빨간 점(환자)이 파란 점(정상)에 파묻혀 있다.
# 리포트의 '1' 클래스에 대한 Recall(재현율)을 보라. 매우 낮거나 0일 것이다.
# 모델은 빨간 점을 무시하고 경계선을 그어버렸다.
```


> 📖 문제 풀이 가이드
>
> 데이터 불균형(Imbalance)의 심각성을 눈으로 확인합니다.
> - 상황: 1000명 중 환자는 50명(5%), 정상인 950명(95%)인 극단적 상황.
> - 시각화: 파란 점(정상)의 바다 속에 빨간 점(환자)이 몇 개 파묻혀 있습니다.
> - 결과: 모델은 빨간 점을 무시해버립니다. 재현율(Recall)이 처참하게 낮게 나옵니다.



In [ ]:
# do write code


## 6.3. \[예제 2\] 무작위 언더 샘플링 (Random Undersampling)

다수 클래스(정상)를 무작위로 삭제하여 비율을 1:1로 맞춘다.

``` python
from imblearn.under_sampling import RandomUnderSampler

# 1. 언더 샘플러 객체 생성
# sampling_strategy='auto': 소수 클래스 개수에 맞춰 다수를 깎아냄 (1:1)
rus = RandomUnderSampler(random_state=42)

# 2. 샘플링 적용 (fit_resample)
X_resampled, y_resampled = rus.fit_resample(X, y)

# 3. 결과 확인
print(f"원본 데이터 크기: {len(y)}")
print(f"원본 1의 개수: {sum(y)}")
print("-" * 30)
print(f"언더 샘플링 후 크기: {len(y_resampled)}")
print(f"샘플링 후 1의 개수: {sum(y_resampled)}")

# 4. 시각화
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_resampled[:, 0], y=X_resampled[:, 1], hue=y_resampled, palette=['blue', 'red'], s=50)
plt.title("After Random Undersampling (Balanced)")
plt.show()

# 설명:
# 파란 점(정상)의 개수가 확 줄어들어 빨간 점과 동등해졌다.
# 학습 속도는 빠르지만, 사라진 파란 점들의 정보를 잃었다.
```


> 📖 문제 풀이 가이드
>
> 다수를 깎아내는 언더 샘플링(Under-sampling)을 시도합니다.
> - 방법: 950명의 정상인 중 무작위로 50명만 남기고 900명을 버립니다.
> - 장점: 1:1 비율이 맞춰지고, 데이터가 줄어서 학습이 엄청 빠릅니다.
> - 단점: 버려진 900명의 정보는 영영 사라집니다. (정보 손실)



In [ ]:
# do write code


## 6.4. \[예제 3\] 무작위 오버 샘플링 (Random Oversampling)

소수 클래스(환자)를 무작위로 복제하여 비율을 1:1로 맞춘다.

``` python
from imblearn.over_sampling import RandomOverSampler

# 1. 오버 샘플러 객체 생성
ros = RandomOverSampler(random_state=42)

# 2. 샘플링 적용
X_over, y_over = ros.fit_resample(X, y)

# 3. 결과 확인
print(f"오버 샘플링 후 크기: {len(y_over)}")
print(f"샘플링 후 0의 개수: {len(y_over) - sum(y_over)}")
print(f"샘플링 후 1의 개수: {sum(y_over)}")

# 4. 시각화
plt.figure(figsize=(8, 6))
# alpha(투명도)를 낮춰서 점이 겹쳐 있음을 표현
sns.scatterplot(x=X_over[:, 0], y=X_over[:, 1], hue=y_over, palette=['blue', 'red'], s=50, alpha=0.3)
plt.title("After Random Oversampling (Points Overlapped)")
plt.show()

# 설명:
# 빨간 점의 개수가 늘어나 파란 점과 같아졌다.
# 하지만 그래프상으로는 위치가 똑같아서 겹쳐 보인다(진한 빨간색).
# 모델은 이 위치를 강제로 외우게 되어 과적합 위험이 있다.
```


> 📖 문제 풀이 가이드
>
> 소수를 복사하는 오버 샘플링(Over-sampling)을 시도합니다.
> - 방법: 50명의 환자 데이터를 19배 복사해서 950명으로 뻥튀기합니다.
> - 시각화: 점들이 겹쳐서 진한 빨간색이 됩니다.
> - 단점: 모델이 똑같은 문제만 계속 풀어서, 그 환자들만 달달 외워버립니다. (과적합)



In [ ]:
# do write code


## 6.5. \[예제 4\] SMOTE: 가상의 환자 생성하기

단순 복제가 아니라, 데이터를 합성하여 분포를 채워주는 SMOTE 기법을
적용한다.

``` python
from imblearn.over_sampling import SMOTE

# 1. SMOTE 객체 생성
smote = SMOTE(random_state=42)

# 2. 샘플링 적용
X_smote, y_smote = smote.fit_resample(X, y)

# 3. 시각화 비교
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, palette=['blue', 'red'])
plt.title("Original Data")

plt.subplot(1, 2, 2)
sns.scatterplot(x=X_smote[:, 0], y=X_smote[:, 1], hue=y_smote, palette=['blue', 'red'])
plt.title("After SMOTE")

plt.show()

# 설명:
# 오른쪽 그림을 보면, 원래 빨간 점들 사이에 없던 새로운 빨간 점들이 생겨나서
# 빨간 점들의 영역이 더 촘촘하고 넓어진 것을 볼 수 있다.
# 이것이 '데이터 합성'의 효과다.
```


> 📖 문제 풀이 가이드
>
> 단순 복제의 단점을 보완한 SMOTE 기법을 사용합니다.
> - 방법: 환자 A와 환자 B 사이에 선을 긋고, 그 선 위에 새로운 가상 환자 C를 만듭니다.
> - 효과: 똑같은 데이터가 아니라 '그럴싸한 새 데이터'가 추가되므로, 과적합을 막으면서 학습 데이터를 확보할 수 있습니다. 헬스케어의 표준입니다.



In [ ]:
# do write code


## 6.6. \[예제 5\] Class Weight: 데이터 조작 없이 공정하게

데이터를 늘리거나 줄이지 않고, 모델 자체의 옵션(`class_weight`)만으로
성능을 개선해본다.

``` python
# 1. 가중치를 적용한 모델 생성
# class_weight='balanced': 데이터 비율에 반비례하게 가중치 자동 설정
# (적은 데이터는 가중치 높게, 많은 데이터는 낮게)
model_weighted = LogisticRegression(class_weight='balanced')

# 2. 원본 데이터로 학습 (샘플링 안 함!)
model_weighted.fit(X, y)
y_pred_weighted = model_weighted.predict(X)

# 3. 결과 비교
print("=== Class Weight 적용 모델 성능 ===")
print(confusion_matrix(y, y_pred_weighted))
print(classification_report(y, y_pred_weighted))

# 설명:
# 예제 1(일반 모델)과 비교했을 때, '1' 클래스에 대한 Recall(재현율)이 대폭 상승했다.
# 데이터를 조작하지 않고도 소수 클래스를 잘 찾아내는 모델이 되었다.
# 다만, 그만큼 정상인(0)을 환자로 오해하는 경우(FP)도 늘어날 수 있다.
```


> 📖 문제 풀이 가이드
>
> 데이터를 건드리지 않고, 모델의 채점 기준(Class Weight)을 바꿉니다.
> - 원리: 정상인을 틀리면 벌점 1점, 환자를 틀리면 벌점 20점!
> - 설정: `class_weight='balanced'`라고 적어주면 알아서 비율을 계산해 적용합니다.
> - 효과: 모델이 벌점을 안 받으려고 소수 클래스(환자)를 필사적으로 맞추려 노력하게 됩니다.



In [ ]:
# do write code
